# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [ ]:
# imports
import os
from isicarchive.api import IsicApi

# Settings (please adapt to your needs!)
username = 'weberj3@mskcc.org'

# DO NOT PUT A PASSWORD into the document, it will be requested,
# or you can store it in a privately readable ~/.netrc file!

doc_folder = 'Z:\\10.Imaging Informatics\\'
# doc_folder = '/Data/MSKCC/'
cache_folder = doc_folder + 'ISIC' + os.sep + 'cache'

# setting debug flag, this will print out every web-API operation
debug = False

# Preparation
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
# caching images and segmentations; this can take a long time!!
# so you may want to skip this cell for now...
api.cache_images()
api.cache_segmentations()

In [ ]:
# list available studies (depends on user authentication!)
api.studies

In [ ]:
# show the number of images in each study
for (study_name, study_id) in api.studies.items():
    print('Study "' + study_name + '" has ' + str(len(api._studies[study_id]['images'])) + ' images.')

In [ ]:
# load a study,
# add the meta-data file (exemplar feature in CSV) as exemplar meta_data,
# and cache the images associated with it
study_folder = doc_folder + 'EASY' + os.sep + 'PILOT' + os.sep
if not os.path.exists(study_folder):
    os.mkdir(study_folder)
study = api.study('ISIC Annotation Study - All Features', exemplar=study_folder + 'pilot.csv')
study.cache_image_data()

In [ ]:
# alternatively, manually load additional meta data later
study.load_meta_data(study_folder + 'study1.csv', dict_key='ISIC_ID',
    list_to_dict=True, meta_key='exemplar', extract_key='feature')

In [ ]:
# load the annotation data
study.load_annotations()

In [ ]:
# select only from users that completed the study
num_images = len(study.images)
users = [u for (u,c) in study.user_completion.items() if c==num_images]
study.select_annotations(users=users)
user_names = {u['_id']: u['name'].replace('User ', '') for u in study.users}

# number of all annotations (across users, images, features)
total_features_annotations = sum(
    [len(a.features) for a in study.annotation_selection.values()])

# determine which features were used
selected_features = dict()
for annotation in study.annotation_selection.values():
    for feature in annotation.features:
        selected_features[feature] = True

# and create necessary lists
full_features_list = sorted(list([f['id'] for f in study.features]))
features_list = sorted(selected_features.keys())
num_features = len(features_list)
features_idx = dict()
for (feat_idx, feat_name) in enumerate(features_list):
    features_idx[feat_name] = feat_idx
category_list = sorted(list(set([v.split(' : ')[0] for v in features_list])))
num_categories = len(category_list)
category_idx = dict()
for (cat_idx, cat_name) in enumerate(category_list):
    category_idx[cat_name] = cat_idx

In [ ]:
# get annotation list
study.select_annotations(images=['ISIC_0016094'], users=users)
annotation_list = list(study.annotation_selection.values())
annotation_list[0].compute_areas()
len(annotation_list[0].features['Lines : Streaks']['lst'])

In [ ]:
# create heatmap with default settings
(heatmap, stats) = study.image_heatmap('ISIC_0016094',
    mix_colors=False,underlay_gray=0.8,users=['578e64b09fc3c10d6fd12e4f'])
api.display_image(heatmap)

In [ ]:
# display stats on the annotation features
stats['featcols']

In [ ]:
# load a dataset
dataset = api.dataset('HAM10000')
dataset

In [ ]:
# load image information into the dataset
dataset.load_images()

In [ ]:
# select images from a dataset
selection = api.select_images(['dataset.name', '==', 'Lymphoma_250_bt'])

# and create a meta-dictionary as a CSV
md = api.write_csv(doc_folder + 'Lymphoma_250_bt_metadata.csv', api.selected_metadata())

In [ ]:
from isicarchive import imfunc
from matplotlib import pyplot as plt
import numpy
# load image's superpixel information and map array
image = api.image('ISIC_0065178')
image.load_image_data()
gray_d = imfunc.image_gray(image.data)
gray_l = imfunc.image_gray(image.data, conv_type='l')
gray_a = imfunc.image_gray(image.data, conv_type='a')
plt.figure(figsize=(16,16))
plt.imshow(numpy.concatenate((numpy.concatenate((image.data, gray_a), axis=1), numpy.concatenate((gray_d, gray_l), axis=1)), axis=0))
plt.show()

In [ ]:
api.write_image(ci, 'C:\\Users\\weberj3\\Documents\\ISIC_0016102_SP0476_n1.png')

In [ ]:
import numpy
from isicarchive.sampler import Sampler
s = Sampler()
numpix = ci.shape[0] * ci.shape[1]
cim = numpy.zeros(numpix).reshape((ci.shape[0], ci.shape[1],))
cim[ci.shape[0]//2, ci.shape[1]//2] = 1
cims = s.sample_grid(cim, 1.0, 'gauss84')
cims = cims / numpy.amax(cims)
cis = imfunc.image_smooth_fft(ci, 84.0)
cimix = imfunc.image_mix(cis, ci, cims)
plt.figure(figsize=(10,10))
plt.imshow(cis)
plt.show()

In [ ]:
cim.shape

In [ ]:
# create superpixel outlines as SVG paths
superpixel_colors = api.superpixel_colors()
outline_svg = image.superpixel_outlines(
    out_format='osvg', path_attribs=superpixel_colors)
outlines = image.superpixel_outlines(
    out_format='osvgp', path_attribs=superpixel_colors)

# display one path
outlines[0]

In [ ]:
# retrieve segmentation (supports image name, image ID (via additional request), and segmentation ID)
image = api.image('ISIC_0016094')
segmentation = api.segmentation('ISIC_0016094')

In [ ]:
# load data
image.load_image_data()
image.load_superpixels()
image.map_superpixels()
segmentation.load_mask_data()
print(str(segmentation.area) + ' pixels selected (' + str(segmentation.area_pct) + '%)')

In [ ]:
l = segmentation.superpixels_in_mask()
print(l)

In [ ]:
# get superpixels in image mask
sp_weights = segmentation.superpixels_in_mask()
sp_half_in = segmentation.superpixels_in_mask(0.5)
sp_full_in = segmentation.superpixels_in_mask(1.0)

# print sum
print('Number of superpixels: ' + str(len(sp_weights)))
print('Average coverage: ' + str(sum(sp_weights) / len(sp_weights)))
print('Superpixels at least half in mask: ' + str(len(sp_half_in)))
print('Superpixels fully in mask: ' + str(len(sp_full_in)))

In [ ]:
a[0].overlap('Vessels : Arborizing', a[1], 'Vessels : Arborizing', 'smcc')

In [ ]:
m0 = a[0].masks['Vessels : Arborizing']
m1 = a[1].masks['Vessels : Arborizing']
import numpy
m0s = m0.shape
m0 = m0.reshape((m0s[0], m0s[1], 1,))
m1 = m1.reshape((m0s[0], m0s[1], 1,))
from isicarchive import imfunc
m0 = imfunc.image_smooth_fft(m0, 0.05)
m1 = imfunc.image_smooth_fft(m1, 0.05)
m2 = numpy.minimum(m0,m1)
m = numpy.concatenate((m0-(0.5*m2).astype(numpy.uint8),m1-(0.5*m2).astype(numpy.uint8),m2), axis=2)
imfunc.image_corr(m0,m1)

In [ ]:
selection = api.select_images([['dataset.name', '==', dataset.name],['meta.clinical.benign_malignant','==','benign']])
len(selection)

In [ ]:
simages = list(selection.keys())

In [ ]:
from isicarchive import func
simages_x = func.getxattr(list(selection.values()), '[].meta.acquisition.pixelsX')
simages_y = func.getxattr(list(selection.values()), '[].meta.acquisition.pixelsY')

In [ ]:
from isicarchive import imfunc
import numpy
mi = numpy.zeros((4500,6000,3), dtype=numpy.uint8)

In [ ]:
import json
from isicarchive import api as iapi

# load a dataset
dataset = api.dataset('HAM10000')
dataset.load_images()
image = api.image(dataset.images[0])
image

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import mpl_toolkits.mplot3d.proj3d as p3d

import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x, y = np.random.rand(2, 100) * 4
hist, xedges, yedges = np.histogram2d(x, y, bins=4, range=[[0, 4], [0, 4]])

# Construct arrays for the anchor positions of the 16 bars.
xpos, ypos = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Construct arrays with the dimensions for the 16 bars.
dx = dy = 0.99 * np.ones_like(zpos)
dz = hist.ravel()

ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')

plt.show()